https://www.gov.br/inep/pt-br/areas-de-atuacao/pesquisas-estatisticas-e-indicadores/censo-da-educacao-superior/resultados

In [8]:
import pandas as pd
import seaborn as sns
from Levenshtein import distance

### Colunas

- NU_ANO_CENSO: Ano do Censo

- NO_REGIAO: Nome da região geográfica do local de oferta do curso

- SG_UF: Sigla da Unidade da Federação do local de oferta do curso

- NO_MUNICIPIO: Nome do Município do local de oferta do curso

- CO_MUNICIPIO: Código do IBGE do municipio do local de oferta do curso

- IN_CAPITAL: Informa se o local de oferta do curso está localizado em capital de Estado
	- 0. Não
	- 1. Sim
	- (.) Não aplicável (Curso a distância)

- TP_REDE: Rede de Ensino
	- 1. Pública
	- 2. Privada

- NO_CINE_ROTULO: Nome do curso, conforme adaptação da Classificação Internacional Normalizada da Educação Cine/Unesco

- TP_GRAU_ACADEMICO: Tipo do grau acadêmico conferido ao ao aluno pela conclusão dos requisitos exigidos pelo curso
	- 1. Bacharelado
	- 2. Licenciatura
	- 3. Tecnológico
	- 4. Bacharelado e Licenciatura
	- (.) Não aplicável (cursos com nivel acadêmico igual a sequencial de formação específica ou  cursos de área básica de curso identificada pela variável TP_ATRIBUTO_INGRESSO)

- TP_MODALIDADE_ENSINO: Tipo da modalidade de ensino do curso
	- 1. Presencial
	- 2. Curso a distância

- TP_NIVEL_ACADEMICO: Tipo do nível acadêmico do curso
	- 1. Graduação
	- 2. Seqüencial de Formação Específica

- QT_ING: Quantidade de ingressante no curso

- QT_ING_FEM: Quantidade de ingressante do sexo feminino no curso

- QT_CONC: Quantidade de concluinte no curso

- QT_CONC_FEM: Quantidade de concluinte no curso do sexo feminino

In [2]:
path_cursos = 'data/cursos.csv.zip'
df_cursos = pd.read_csv(path_cursos)
df_cursos

,NU_ANO_CENSO,NO_REGIAO,SG_UF,NO_MUNICIPIO,CO_MUNICIPIO,IN_CAPITAL,TP_REDE,NO_CINE_ROTULO,TP_GRAU_ACADEMICO,TP_MODALIDADE_ENSINO,TP_NIVEL_ACADEMICO,QT_ING,QT_ING_FEM,QT_CONC,QT_CONC_FEM
0,2009,Centro-Oeste,DF,Brasília,5300108.0,1.0,1,Ciência da computação,1.0,1,1,90.0,14.0,42.0,4.0
1,2009,Centro-Oeste,DF,Brasília,5300108.0,1.0,2,Sistemas de informação,3.0,1,1,56.0,4.0,0.0,0.0
2,2009,Centro-Oeste,DF,Brasília,5300108.0,1.0,2,Sistemas de informação,3.0,1,1,95.0,25.0,1.0,1.0
3,2009,Centro-Oeste,DF,Brasília,5300108.0,1.0,2,Sistemas de informação,1.0,1,1,37.0,12.0,11.0,2.0
4,2009,Centro-Oeste,DF,Brasília,5300108.0,1.0,2,Sistemas de informação,1.0,1,1,2.0,0.0,9.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92467,2020,NaN,NaN,NaN,NaN,NaN,2,Sistemas de informação,3.0,2,1,0.0,0.0,0.0,0.0
92468,2020,NaN,NaN,NaN,NaN,NaN,2,Jogos digitais,3.0,2,1,0.0,0.0,0.0,0.0
92469,2020,NaN,NaN,NaN,NaN,NaN,2,Sistemas de informação,3.0,2,1,0.0,0.0,0.0,0.0
92470,2020,NaN,NaN,NaN,NaN,NaN,1,Ciência de dados,1.0,2,1,0.0,0.0,0.0,0.0


In [3]:
df_erros = (df_cursos
	[
		(df_cursos['NO_MUNICIPIO'].notna())
		&
		(df_cursos['NO_MUNICIPIO'].str.contains('�'))
	]['NO_MUNICIPIO'].value_counts())
df_erros

Bel�m                  668
São Jos� dos Campos    553
Florian�polis          521
Macei�                 484
Jo�o Pessoa            466
                      ... 
Concei��o dos Ouros      1
Mendon�a                 1
Andara�                  1
Vila Pav�o               1
Santa Cec�lia            1
Name: NO_MUNICIPIO, Length: 781, dtype: int64

In [5]:
url_mun = 'https://raw.githubusercontent.com/mapaslivres/municipios-br/main/tabelas/municipios.csv'
df_mun = pd.read_csv(url_mun)
df_mun

,municipio,uf,uf_code,name,mesoregion,microregion,rgint,rgi,osm_relation_id,wikidata_id,is_capital,wikipedia_pt,lon,lat,no_accents,slug_name,alternative_names,pop_21
0,1100015,11,RO,Alta Floresta d'Oeste,1102,11006,1102,110005,325842,Q1761456,NaN,Alta Floresta d'Oeste,-61.999824,-11.935540,Alta Floresta d'Oeste,alta-floresta-doeste,NaN,22516
1,1100023,11,RO,Ariquemes,1102,11003,1101,110002,325888,Q1646975,NaN,Ariquemes,-63.033269,-9.908463,Ariquemes,ariquemes,NaN,111148
2,1100031,11,RO,Cabixi,1102,11008,1102,110006,325882,Q940433,NaN,Cabixi,-60.544314,-13.499763,Cabixi,cabixi,NaN,5067
3,1100049,11,RO,Cacoal,1102,11006,1102,110005,325872,Q1761637,NaN,Cacoal,-61.442944,-11.433865,Cacoal,cacoal,NaN,86416
4,1100056,11,RO,Cerejeiras,1102,11008,1102,110006,325883,Q1803210,NaN,Cerejeiras,-60.818426,-13.195033,Cerejeiras,cerejeiras,NaN,16088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,5222005,52,GO,Vianópolis,5205,52016,5201,520002,334495,Q22067073,NaN,Vianópolis,-48.513471,-16.742081,Vianopolis,vianopolis,NaN,14088
5566,5222054,52,GO,Vicentinópolis,5205,52015,5202,520009,334491,Q940869,NaN,Vicentinópolis,-49.807239,-17.732662,Vicentinopolis,vicentinopolis,NaN,9002
5567,5222203,52,GO,Vila Boa,5204,52012,5206,520022,334493,Q1025753,NaN,Vila Boa (Goiás),-47.057361,-15.034889,Vila Boa,vila-boa,NaN,6451
5568,5222302,52,GO,Vila Propício,5204,52012,5205,520018,334444,Q989519,NaN,Vila Propício,-48.882422,-15.454635,Vila Propicio,vila-propicio,NaN,5941


In [32]:
muns = df_mun['name'].unique()
names = dict()
for mun_err in df_erros.index:
	for mun in muns:
		if distance(mun_err, mun) == 1 and len(mun) == len(mun_err):
			try:
				names[mun_err].append(mun)
			except:
				names[mun_err] = [mun]

In [33]:
dict_map = dict()
for mun, ops in names.items():
	if len(ops) == 1:
		dict_map[mun] = ops[0]

In [35]:
dict_map

{'Bel�m': 'Belém',
 'São Jos� dos Campos': 'São José dos Campos',
 'Florian�polis': 'Florianópolis',
 'Macei�': 'Maceió',
 'Jo�o Pessoa': 'João Pessoa',
 'Ribeir�o Preto': 'Ribeirão Preto',
 'Uberl�ndia': 'Uberlândia',
 'Santo Andr�': 'Santo André',
 'Vit�ria': 'Vitória',
 'São Lu�s': 'São Luís',
 'Jundia�': 'Jundiaí',
 'Niter�i': 'Niterói',
 'São Jos� do Rio Preto': 'São José do Rio Preto',
 'Maring�': 'Maringá',
 'Macap�': 'Macapá',
 'Nova Igua�u': 'Nova Iguaçu',
 'Chapec�': 'Chapecó',
 'Foz do Igua�u': 'Foz do Iguaçu',
 'Vit�ria da Conquista': 'Vitória da Conquista',
 'São Jos�': 'São José',
 'Petr�polis': 'Petrópolis',
 'Ara�atuba': 'Araçatuba',
 'Marab�': 'Marabá',
 'Rondon�polis': 'Rondonópolis',
 'Santar�m': 'Santarém',
 'Taubat�': 'Taubaté',
 'Mau�': 'Mauá',
 'Bento Gon�alves': 'Bento Gonçalves',
 'Carapicu�ba': 'Carapicuíba',
 'Crici�ma': 'Criciúma',
 'Jacare�': 'Jacareí',
 'São Gon�alo': 'São Gonçalo',
 'Mar�lia': 'Marília',
 'Jaragu� do Sul': 'Jaraguá do Sul',
 'São Jos� dos

In [36]:
df_cursos['NO_MUNICIPIO']

0        Brasília
1        Brasília
2        Brasília
3        Brasília
4        Brasília
           ...   
92467         NaN
92468         NaN
92469         NaN
92470         NaN
92471         NaN
Name: NO_MUNICIPIO, Length: 92472, dtype: object

In [38]:
df_cursos['NO_MUNICIPIO'] = df_cursos['NO_MUNICIPIO'].apply(
	lambda x: dict_map[x] if x in dict_map else x)

In [40]:
df_cursos.to_csv('data/cursos.csv.zip', index=False)